__tests__ for `importnb`

In [1]:
    from importnb import Notebook, reload, parameterize, Parameterize
    from importnb.remote import Remote
    from pytest import fixture, raises, mark
    import json, linecache, inspect, ast, sys, io
    from pathlib import Path
    import contextlib
    

In [2]:
    try: from IPython import get_ipython
    except: get_ipython = lambda: None
    

## Marks

In [3]:
    ipy = mark.skipif(not get_ipython(), reason="""Not IPython.""")
    py34 = sys.version_info.major == 3 and sys.version_info.minor == 4

In [4]:
    def test_basic():
        with Notebook(): 
            import foobar
            reload(foobar)
        assert isinstance(foobar.__loader__, Notebook)
        assert foobar.__test__
        del foobar

In [5]:
    @fixture
    def module():
        sys.path_importer_cache.clear()
        with Notebook(): 
            import foobar
            yield reload(foobar)
        del foobar

In [6]:
    @fixture
    def package():
        with Notebook(): 
            import foobaz.foobar
        yield foobaz.foobar
        del foobaz

In [7]:
    def test_package(package, module):
        assert not module.__package__
        assert package.__package__

In [8]:
    def test_reload(module): 
        """delete a method from the module and see if it is recovered"""
        del module._repr_markdown_
        
        """The contextmanager is required."""
        with Notebook():
            reload(module)
        assert hasattr(module, '_repr_markdown_')

In [9]:
    def test_docstrings(module):
        assert module.__doc__
        assert module.function_with_markdown_docstring.__doc__
        assert module.function_with_python_docstring.__doc__

In [10]:
    def test_docstring_opts(module):
        with Notebook(markdown_docstring=False):
            reload(module)
        assert module.__doc__
        assert not module.function_with_markdown_docstring.__doc__
        assert module.function_with_python_docstring.__doc__

In [11]:
    def test_from_file(module):
        new = Notebook.load(module.__file__)
        assert module is not new

In [12]:
    @mark.skipif(py34, reason="There is no py34 lazyloader.")
    def test_lazy(capsys):
        """Use stdout to test this depsite there probably being a better way"""
        with Notebook(lazy=True): 
            module = reload(__import__('lazy_test'))
            assert not capsys.readouterr()[0], capsys.readouterr()[0]
            module.foo, "The function gets executed here"
            assert capsys.readouterr()[0]

In [13]:
    def test_module_source(module): 
        with raises(getattr(json, 'JSONDecodeError', ValueError)):
            json.loads(''.join(linecache.cache[module.__file__][2]))
        assert inspect.getsource(module).strip() == ''.join(linecache.cache[module.__file__][2]).strip()

In [14]:
    import sys

In [15]:
    @mark.skipif(py34, reason="I don't know why this fails on 3.4.")
    def test_main():
        with Notebook('__main__'):
            try: del sys.modules['foobar']
            finally: import foobar
            
        assert foobar.__name__ == '__main__'

In [16]:
    def test_object_source(module): 
        assert ast.parse(inspect.getsource(module.function_with_markdown_docstring)), """The source is invalid"""

In [17]:
    def test_python_file():
        import foobar as module
        assert reload(module).__file__.endswith('.py'), """Python didn't take precedent."""
        with Notebook(): assert reload(module).__file__.endswith('.ipynb')
        assert reload(module).__file__.endswith('.py')
        with Notebook(): assert reload(module).__file__.endswith('.ipynb')

In [18]:
    @ipy
    def test_cli(module): 
        __import__('subprocess').check_call(
            'ipython -m {}'.format(module.__name__).split(), cwd=str(Path(module.__file__).parent))
        __import__('subprocess').check_call(
            'ipython -m importnb -- {}'.format(module.__file__).split(), cwd=str(Path(module.__file__).parent))

In [19]:
    def test_parameterize(module):
        f = parameterize(module)
        assert 'a_parameter' in f.__signature__.parameters
        assert 'not_a_parameter' not in f.__signature__.parameters
        assert isinstance(f(), type(module))

In [20]:
    def test_minified_json(module): 
        with open(module.__file__) as f, open('foobarmin.ipynb', 'w') as o: 
            json.dump(json.load(f), o, indent=None)
            
        with Notebook():
            import foobarmin 
            
        assert inspect.getsource(foobarmin.function_with_markdown_docstring)
            
        with open(foobarmin.__file__) as file:
            assert json.load(file)

In [21]:
    def test_fuzzy_finder(): 
        import __bar
        assert __bar.__name__ == 'foobar'
        del __bar

In [22]:
    def test_remote(monkeypatch):
        def mocked(url):
            class DummyNotebook(object):
                def read(self):
                    with open('tests/foobar.ipynb', 'rb') as file: return file.read()
            return DummyNotebook()
        monkeypatch.setattr(__import__('urllib').request, "urlopen", mocked)
        with Remote('http://0.0.0.0:8888/*.ipynb'): 
            module = __import__('tests/foobar')
        assert module.__file__.startswith('http')
        assert module.function_with_markdown_docstring

`importnb` should be able to transform expressions beginning with magics.

    def test_magic_syntax():...